# Diffuse Proteins, Design and Predict Their Sequences

### Setup

In [2]:
import protslurm
from protslurm.poses import Poses
from protslurm.runners.rfdiffusion import RFdiffusion

In [4]:
# first, we define the jobstarters that should be used throughout the script (this can be changed here to slurm if needed)
cpu_jobstarter = protslurm.jobstarters.LocalJobStarter(max_cores=7)
gpu_jobstarter = protslurm.jobstarters.LocalJobStarter(max_cores=1)

# on cluster, gpu_jobstarter looks very different:
sbatch_gpu_jobstarter = protslurm.jobstarters.SbatchArrayJobstarter(max_cores=10, gpus=1)

# we setup an empty Poses object
proteins = Poses(
    poses=None,
    work_dir="./rfdiffusion_mpnn_esm_output/",
    storage_format="pickle",
    jobstarter=cpu_jobstarter
)

print(proteins.df)

Empty DataFrame
Columns: [input_poses, poses, poses_description]
Index: []


# Creating De Novo Proteins from Scratch with RFDiffusion

In [ ]:
# setup runner.
rfdiffusion_runner = RFdiffusion(jobstarter=gpu_jobstarter)

# start runner
diffused_proteins = rfdiffusion_runner.run(
    poses=proteins,
    output_dir="rfdiffusion_mpnn_esm_output/general_diffusion/"
)